Merge dataset

In [19]:
import pandas as pd


In [20]:
df1 = pd.read_csv('data/raw/porn-with-dates-2022.csv')
df2 = pd.read_csv('data/raw/output_async.csv')

/tmp/ipykernel_2209746/3463678343.py:2: DtypeWarning: Columns (1,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('data/raw/output_async.csv')


In [22]:
print("Columns in df1:", df1.columns.tolist())
print("Columns in df2:", df2.columns.tolist())


Columns in df1: ['url', 'title', 'categories', 'date', 'views', 'upvotes', 'downvotes', 'production', 'comments', 'actors']
Columns in df2: ['url', '_upload_date', '_votes_up', '_views', '_categories', '_tags', '_title']


In [23]:
df2_renamed = df2.rename(columns={
    '_title': 'title',
    '_categories': 'categories',
    '_upload_date': 'date',
    '_views': 'views',
    '_votes_up': 'upvotes',
        })


In [24]:
print("Columns in df1:", df1.columns.tolist())
print("Columns in df2_renamed:", df2_renamed.columns.tolist())

Columns in df1: ['url', 'title', 'categories', 'date', 'views', 'upvotes', 'downvotes', 'production', 'comments', 'actors']
Columns in df2_renamed: ['url', 'date', 'upvotes', 'views', 'categories', '_tags', 'title']


In [25]:
metadata_cols = ['title', 'categories', 'date', 'views', 'upvotes']
df2_renamed.replace('', pd.NA, inplace=True)
# Drop rows where all specified metadata fields are missing
df2_renamed_cleaned = df2_renamed.dropna(subset=metadata_cols, how='all')
print(f"Original rows: {len(df2_renamed)}")
print(f"Remaining after cleaning: {len(df2_renamed_cleaned)}")


Original rows: 434600
Remaining after cleaning: 38122


In [26]:
combined_df = pd.concat([df1, df2_renamed_cleaned], ignore_index=True, sort=False)


In [27]:
combined_df.columns


Index(['url', 'title', 'categories', 'date', 'views', 'upvotes', 'downvotes',
       'production', 'comments', 'actors', '_tags'],
      dtype='object')

In [29]:
combined_df.categories

0         ['Babe', 'Big Tits', 'Brunette', 'HD Porn', 'H...
1         ['Blowjob', 'HD Porn', 'Hardcore', 'MILF', 'Ro...
2         ['Big Tits', 'Brunette', 'Cumshot', 'HD Porn',...
3         ['Amateur', 'Anal', 'Creampie', 'HD Porn', 'Ha...
4         ['Amateur', 'Blonde', 'Brunette', 'Fingering',...
                                ...                        
256121                                                  NaN
256122    Anal;Blowjob;Bondage;Brunette;Creampie;HD Porn...
256123    Amateur;Big Tits;Blonde;College (18+);Ebony;Ex...
256124    Amateur;Anal;Big Dick;Exclusive;Female Orgasm;...
256125    18-25;Amateur;Behind The Scenes;Female Orgasm;...
Name: categories, Length: 256126, dtype: object

In [30]:
import ast

def normalize_categories(val):
    if pd.isna(val):
        return []
    if isinstance(val, list):
        return val
    try:
        # Try to parse stringified list
        parsed = ast.literal_eval(val)
        if isinstance(parsed, list):
            return parsed
    except (ValueError, SyntaxError):
        pass
    # Fallback: split on semicolon
    return [x.strip() for x in val.split(';')]

combined_df['categories'] = combined_df['categories'].apply(normalize_categories)


In [31]:
combined_df.categories


0         [Babe, Big Tits, Brunette, HD Porn, Handjob, T...
1         [Blowjob, HD Porn, Hardcore, MILF, Rough Sex, ...
2         [Big Tits, Brunette, Cumshot, HD Porn, Latina,...
3         [Amateur, Anal, Creampie, HD Porn, Hardcore, POV]
4         [Amateur, Blonde, Brunette, Fingering, HD Porn...
                                ...                        
256121                                                   []
256122    [Anal, Blowjob, Bondage, Brunette, Creampie, H...
256123    [Amateur, Big Tits, Blonde, College (18+), Ebo...
256124    [Amateur, Anal, Big Dick, Exclusive, Female Or...
256125    [18-25, Amateur, Behind The Scenes, Female Org...
Name: categories, Length: 256126, dtype: object

In [32]:
combined_df['date'] = pd.to_datetime(combined_df['date'], errors='coerce')
min_date = combined_df['date'].min()
max_date = combined_df['date'].max()

print(f"Date range: {min_date.date()} to {max_date.date()}")



Date range: 2008-01-01 to 2024-08-08


In [33]:
combined_df['year'] = combined_df['date'].dt.year
year_counts = combined_df['year'].value_counts().sort_index()
print(year_counts)


year
2008.0      763
2009.0     9051
2010.0     8413
2011.0     6942
2012.0    11053
2013.0    24605
2014.0    27111
2015.0    25599
2016.0    27891
2017.0    29525
2018.0    52746
2019.0     2557
2020.0     4557
2021.0     6608
2022.0     5947
2023.0     6778
2024.0     2837
Name: count, dtype: int64


In [34]:
combined_df.to_csv('data/raw/data2008-2024.csv', index=False)
